In [1]:
# init
import pandas as pd
import h5py
import numpy as np
#import matplotlib.pyplot as plt

X_train = h5py.File("X_train.h5", "r")
y_train = pd.read_csv("y_train.csv")

In [2]:
def extract_features(h5):
    data = h5["features"][:]
    features = []
    features.append(data[:, :])
    #features.append(data[:, :11])
    #features.append(data[:, 11:].max(1).reshape(-1, 1))
    #features.append(data[:, 11:].min(1).reshape(-1, 1))
    #features.append(np.abs(data[:, 11:]).mean(1).reshape(-1, 1))
    #features.append(deltabeta(data))
    ##features.append(nbmax(data[:, 11:]).reshape(-1, 1))
    features = np.concatenate(features, 1)
    return features

# Données
#data = extract_features(X_train)

In [3]:
data = np.load("dataandones.npy")
from tensorflow.keras.utils import to_categorical

y_train = np.squeeze(y_train.values[:, 1])
y_train = to_categorical(y_train, num_classes=None)
#categorical_crossentropy

In [4]:
# RNN for the sequences
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(data[:, 11:], y_train)

In [5]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [6]:
from __future__ import print_function

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM

max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 128

print('Loading data...')
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

#print('Pad sequences (samples x time)')
#x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
#x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, input_shape=(x_test.shape[1], 1)))
model.add(Dense(3, activation='softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Loading data...
196225 train sequences
65409 test sequences
x_train shape: (196225, 1250, 1)
x_test shape: (65409, 1250, 1)
Build model...
Train...
Train on 196225 samples, validate on 65409 samples
Epoch 1/15
196225/196225 [==============================] - 2357s 12ms/sample - loss: 1.0474 - accuracy: 0.4603 - val_loss: 1.0579 - val_accuracy: 0.4572
Epoch 2/15
196225/196225 [==============================] - 2339s 12ms/sample - loss: 1.0449 - accuracy: 0.4617 - val_loss: 1.0460 - val_accuracy: 0.4633
Epoch 3/15
196225/196225 [==============================] - 2347s 12ms/sample - loss: 1.0425 - accuracy: 0.4641 - val_loss: 1.0684 - val_accuracy: 0.4613
Epoch 4/15
196225/196225 [==============================] - 2320s 12ms/sample - loss: 1.0386 - accuracy: 0.4656 - val_loss: 1.0336 - val_accuracy: 0.4722
Epoch 5/15
196225/196225 [==============================] - 2347s 12ms/sample - loss: 1.0357 - accuracy: 0.4663 - val_loss: 1.0345 - val_accuracy: 0.4744
Epoch 6/15
196225/196225 [=====

Test score: 1.0217137069617692
Test accuracy: 0.48560596


In [7]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")
 
# later...
 
## load json and create model
#json_file = open('model.json', 'r')
#loaded_model_json = json_file.read()
#json_file.close()
#loaded_model = model_from_json(loaded_model_json)
## load weights into new model
#loaded_model.load_weights("model.h5")
#print("Loaded model from disk")

Saved model to disk


In [8]:
x_test = h5py.File("X_test.h5", "r")
x_test = extract_features(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [9]:
y_pred = model.predict(x_test[:, 11:], batch_size=batch_size)
print(y_pred[:10])

[[0.38091755 0.34365928 0.2754232 ]
 [0.39327386 0.35142553 0.2553006 ]
 [0.43778297 0.30743852 0.25477856]
 [0.22760092 0.39937147 0.37302765]
 [0.2708699  0.38510686 0.34402323]
 [0.34744352 0.35799345 0.29456303]
 [0.22657877 0.3502935  0.42312777]
 [0.709741   0.17358874 0.11667024]
 [0.724802   0.15342095 0.12177702]
 [0.53110147 0.2758302  0.19306831]]


In [10]:
np.save("y_pred_sequentiel", y_pred)

In [ ]:
with open("y_test.csv", "w") as f:
    f.write("".join(["id,label\n"] + ["{},{}\n".format(i, y) for i, y in enumerate(y_pred)]))